This notebook is used for testing purposes hueristics

In [2]:
import os
import numpy as np
from tqdm import trange, tqdm

In [ ]:
instance_path = '../QAPInstances/'
soln_path     = '../QAPSolns/'

def read_integers(filename):
    with open(filename) as f:
        return [int(elem) for elem in f.read().split()]


def open_solution(filename: str):
    file_it = iter(read_integers(filename))
    _ = next(file_it)    # this is just how the files within the lib are formatted
    return next(file_it)

        

def test_hueristic(qap_hueristic, n_iters=10_000, tai_only=False):
    """
    n_iters: number of iterations on each test instance
    tai_only: if True, restricts testing to the Tai instances, reducing computation time
    """
    
    results = [] # for storing results

    for filename in tqdm(os.listdir(instance_path)):
        
        # skips any instances that are not Tai
        if tai_only and 'tai' not in filename: continue  

        file_it = iter(read_integers(instance_path+filename))

        # open QAP instance param's 
        n = next(file_it)
        w = np.array([[next(file_it) for j in range(n)] for i in range(n)])
        d = np.array([[next(file_it) for j in range(n)] for i in range(n)])

        # generate an instance
        hueristic = qap_hueristic(w, d)

        # open up corresponding soln from QAPLib: 
        soln_file = filename[:-4]+'.sln' # this removes the .dat from filename
        
        try:
            qap_soln = open_solution(soln_path+soln_file)
            huerstic_soln, _, _ = hueristic.soln(n_iters)

            # compute gap
            gap = 100*(hueristic.cost(huerstic_soln) - qap_soln)/qap_soln
            results.append((soln_file, gap))
            

        # any instances without corresponding solution files are deleted
        except FileNotFoundError:
            os.remove(instance_path+filename) 
    
    # just printing results for now, but will need to save to file or something 
    print(results)

    return